In [1]:
import pandas as pd
import json
import configparser
from pathlib import Path
import os

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SimpleRNN

configs = configparser.ConfigParser()
configs.read("configuracoes.conf",encoding='utf-8')
data_folder = Path(configs["DEFAULT"]["folder_path"])
path = data_folder / configs["DEFAULT"]["file"]
objeto = configs["DEFAULT"]["objeto"]
regiao = configs["DEFAULT"]["regiao"]

look_back = 12 #Quantidade de meses anteriores usado para prever
forecast_horizon = int(configs["DEFAULT"]["meses"]) #Quantidade de meses a serem previstos

# Prever Cesta

In [2]:
#Carrega os dados
cesta = pd.read_excel(path)
cesta.drop(['ano'], axis=1, inplace=True)
cesta = cesta / 1000
cesta = cesta.iloc[::, ::]

#Cria sequencias de tempo de tamanho (meses anteriores + meses a serem previstos)
for n_step in range(1, look_back + forecast_horizon):
    cesta[f'preco t(h + {n_step})'] = cesta['preco'].shift(-n_step).values
cesta.dropna(inplace=True)
cesta.reset_index(drop=True, inplace=True)

#Divide os dados em Treino e Teste
X_train = cesta.iloc[::, :look_back:].values
y_train = cesta.iloc[::, look_back::].values
X_val = cesta.iloc[-1::, -12::].values

#remodela os dados para formato (batches, sequencia de tempo, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

#Cria o modelo
model=Sequential()
model.add(SimpleRNN(24, stateful=True, input_shape=(look_back, 1), batch_input_shape=(1, look_back, 1)))
model.add(Dense(forecast_horizon))
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003))

#Treina e prever (forecast_horizon) quantidade de tempo
history = model.fit(X_train, y_train, epochs= 150, batch_size=1, shuffle=False)
forecast = model.predict(X_val, batch_size=1)

resultados = [valor for valor in forecast[0]]
objeto = objeto.replace(" ",'_')
output = {objeto.lower():str(resultados)}
filename = f"previsao_RNN_{objeto.lower()}_{regiao.lower()}.json"
pasta_atual = os.getcwd()
full_url = Path(pasta_atual) / 'resultados' / 'previsoes_cesta' / filename
os.makedirs(os.path.dirname(full_url), exist_ok=True)
with open(full_url,"w") as file:
    json.dump(output,file,ensure_ascii=False)
print("Arquivo com resultados gerados")

Epoch 1/150
198/198 [==============================] - 0s 2ms/step - loss: 0.0032
Epoch 2/150
198/198 [==============================] - 0s 2ms/step - loss: 0.0082
Epoch 3/150
198/198 [==============================] - 0s 2ms/step - loss: 0.0052
Epoch 4/150
198/198 [==============================] - 0s 2ms/step - loss: 0.0036
Epoch 5/150
198/198 [==============================] - 0s 2ms/step - loss: 0.0024
Epoch 6/150
198/198 [==============================] - 0s 1ms/step - loss: 0.0018
Epoch 7/150
198/198 [==============================] - 0s 1ms/step - loss: 0.0013
Epoch 8/150
198/198 [==============================] - 0s 1ms/step - loss: 0.0010
Epoch 9/150
198/198 [==============================] - 0s 1ms/step - loss: 8.7465e-04
Epoch 10/150
198/198 [==============================] - 0s 2ms/step - loss: 7.4983e-04
Epoch 11/150
198/198 [==============================] - 0s 1ms/step - loss: 6.6297e-04
Epoch 12/150
198/198 [==============================] - 0s 1ms/step - loss: 6.0407e-

# Prever Produtos

In [2]:
path_inicial = r'C:\\Users\\mathe\\Documents\\NotebooksMateriais\\previsao_cestas\\datasets_produtos\\'

regioes = ['ilheus','itabuna']
produtos = ['acucar', 'arroz', 'banana', 'cafe', 'carne', 'farinha', 'feijao', 'leite', 'manteiga', 'oleo', 'pao', 'tomate']

for regiao in regioes:
    for objeto in produtos:
        #Carrega os dados
        path = f'{path_inicial}{regiao}/{objeto}_{regiao}.xlsx'
        cesta = pd.read_excel(path)
        cesta.drop(['ano'], axis=1, inplace=True)
        cesta = cesta / 1000
        cesta = cesta.iloc[::]

        #Cria sequencias de tempo de tamanho (meses anteriores + meses a serem previstos)
        for n_step in range(1, look_back + forecast_horizon):
            cesta[f'preco t(h + {n_step})'] = cesta['preco'].shift(-n_step).values
        cesta.dropna(inplace=True)
        cesta.reset_index(drop=True, inplace=True)

        #Divide os dados em Treino e Teste
        X_train = cesta.iloc[::, :look_back:].values
        y_train = cesta.iloc[::, look_back::].values
        X_val = cesta.iloc[-1::, -12::].values

        #remodela os dados para formato (batches, sequencia de tempo, features)
        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

        #Cria o modelo
        model=Sequential()
        model.add(SimpleRNN(24, stateful=True, input_shape=(look_back, 1), batch_input_shape=(1, look_back, 1)))
        model.add(Dense(forecast_horizon))
        model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003))

        #Treina e prever (forecast_horizon) quantidade de tempo
        history = model.fit(X_train, y_train, epochs= 150, batch_size=1, shuffle=False)
        forecast = model.predict(X_val, batch_size=1)

        resultados = [valor for valor in forecast[0]]
        objeto = objeto.replace(" ", '_')
        output = {objeto.lower():str(resultados)}
        filename = f"previsao_RNN_{objeto.lower()}_{regiao.lower()}.json"
        pasta_atual = os.getcwd()
        full_url = Path(pasta_atual) / 'resultados' / 'previsoes_produtos' / f'{regiao}' / filename
        os.makedirs(os.path.dirname(full_url), exist_ok=True)
        with open(full_url,"w") as file:
            json.dump(output,file,ensure_ascii=False)
        print("Arquivo com resultados gerados")

Epoch 1/150
198/198 [==============================] - 0s 1ms/step - loss: 1.3605e-05
Epoch 2/150
198/198 [==============================] - 0s 1ms/step - loss: 7.8893e-06
Epoch 3/150
198/198 [==============================] - 0s 1ms/step - loss: 6.6662e-06
Epoch 4/150
198/198 [==============================] - 0s 1ms/step - loss: 5.3426e-06
Epoch 5/150
198/198 [==============================] - 0s 1ms/step - loss: 4.9948e-06
Epoch 6/150
198/198 [==============================] - 0s 1ms/step - loss: 4.7040e-06
Epoch 7/150
198/198 [==============================] - 0s 1ms/step - loss: 4.4351e-06
Epoch 8/150
198/198 [==============================] - 0s 1ms/step - loss: 4.1306e-06
Epoch 9/150
198/198 [==============================] - 0s 1ms/step - loss: 4.1251e-06
Epoch 10/150
198/198 [==============================] - 0s 1ms/step - loss: 4.5961e-06
Epoch 11/150
198/198 [==============================] - 0s 1ms/step - loss: 3.9163e-06
Epoch 12/150
198/198 [==============================

Epoch 95/150
198/198 [==============================] - 0s 942us/step - loss: 4.2415e-06
Epoch 96/150
198/198 [==============================] - 0s 995us/step - loss: 4.4078e-06
Epoch 97/150
198/198 [==============================] - 0s 965us/step - loss: 4.1623e-06
Epoch 98/150
198/198 [==============================] - 0s 967us/step - loss: 4.0311e-06
Epoch 99/150
198/198 [==============================] - 0s 972us/step - loss: 3.9705e-06
Epoch 100/150
198/198 [==============================] - 0s 970us/step - loss: 4.1032e-06
Epoch 101/150
198/198 [==============================] - 0s 1ms/step - loss: 4.4853e-06
Epoch 102/150
198/198 [==============================] - 0s 1ms/step - loss: 4.3863e-06
Epoch 103/150
198/198 [==============================] - 0s 1ms/step - loss: 4.5562e-06
Epoch 104/150
198/198 [==============================] - 0s 1ms/step - loss: 4.6226e-06
Epoch 105/150
198/198 [==============================] - 0s 1ms/step - loss: 4.6889e-06
Epoch 106/150
198/198 [==

198/198 [==============================] - 0s 1ms/step - loss: 5.9180e-06
Epoch 38/150
198/198 [==============================] - 0s 1ms/step - loss: 5.9043e-06
Epoch 39/150
198/198 [==============================] - 0s 1ms/step - loss: 5.8065e-06
Epoch 40/150
198/198 [==============================] - 0s 1ms/step - loss: 5.6687e-06
Epoch 41/150
198/198 [==============================] - 0s 1ms/step - loss: 5.4876e-06
Epoch 42/150
198/198 [==============================] - 0s 1ms/step - loss: 5.3208e-06
Epoch 43/150
198/198 [==============================] - 0s 1ms/step - loss: 5.2136e-06
Epoch 44/150
198/198 [==============================] - 0s 982us/step - loss: 5.1408e-06
Epoch 45/150
198/198 [==============================] - 0s 1ms/step - loss: 5.1225e-06
Epoch 46/150
198/198 [==============================] - 0s 982us/step - loss: 4.9134e-06
Epoch 47/150
198/198 [==============================] - 0s 1ms/step - loss: 4.9796e-06
Epoch 48/150
198/198 [==============================

198/198 [==============================] - 0s 972us/step - loss: 4.3521e-06
Epoch 130/150
198/198 [==============================] - 0s 982us/step - loss: 4.1027e-06
Epoch 131/150
198/198 [==============================] - 0s 972us/step - loss: 9.2638e-06
Epoch 132/150
198/198 [==============================] - 0s 997us/step - loss: 4.8008e-06
Epoch 133/150
198/198 [==============================] - 0s 986us/step - loss: 4.6618e-06
Epoch 134/150
198/198 [==============================] - 0s 967us/step - loss: 4.4663e-06
Epoch 135/150
198/198 [==============================] - 0s 1ms/step - loss: 4.6679e-06
Epoch 136/150
198/198 [==============================] - 0s 1ms/step - loss: 4.7197e-06
Epoch 137/150
198/198 [==============================] - 0s 1ms/step - loss: 4.4997e-06
Epoch 138/150
198/198 [==============================] - 0s 1ms/step - loss: 4.5936e-06
Epoch 139/150
198/198 [==============================] - 0s 1ms/step - loss: 4.4874e-06
Epoch 140/150
198/198 [===========

198/198 [==============================] - 0s 986us/step - loss: 3.5379e-05
Epoch 72/150
198/198 [==============================] - 0s 1ms/step - loss: 3.5349e-05
Epoch 73/150
198/198 [==============================] - 0s 1ms/step - loss: 3.5319e-05
Epoch 74/150
198/198 [==============================] - 0s 1ms/step - loss: 3.5292e-05
Epoch 75/150
198/198 [==============================] - 0s 985us/step - loss: 3.5266e-05
Epoch 76/150
198/198 [==============================] - 0s 1ms/step - loss: 3.5241e-05
Epoch 77/150
198/198 [==============================] - 0s 990us/step - loss: 3.5219e-05
Epoch 78/150
198/198 [==============================] - 0s 990us/step - loss: 3.5197e-05
Epoch 79/150
198/198 [==============================] - 0s 979us/step - loss: 3.5177e-05
Epoch 80/150
198/198 [==============================] - 0s 990us/step - loss: 3.5158e-05
Epoch 81/150
198/198 [==============================] - 0s 988us/step - loss: 3.5138e-05
Epoch 82/150
198/198 [====================

...

198/198 [==============================] - 0s 986us/step - loss: 1.2894e-06
Epoch 100/150
198/198 [==============================] - 0s 995us/step - loss: 1.3319e-06
Epoch 101/150
198/198 [==============================] - 0s 987us/step - loss: 1.3047e-06 0s - loss: 1.4390e-0
Epoch 102/150
198/198 [==============================] - 0s 992us/step - loss: 1.3720e-06
Epoch 103/150
198/198 [==============================] - 0s 1ms/step - loss: 1.3109e-06
Epoch 104/150
198/198 [==============================] - 0s 992us/step - loss: 1.3358e-06
Epoch 105/150
198/198 [==============================] - 0s 992us/step - loss: 1.9505e-06
Epoch 106/150
198/198 [==============================] - 0s 1ms/step - loss: 1.3012e-06
Epoch 107/150
198/198 [==============================] - 0s 1ms/step - loss: 1.3225e-06
Epoch 108/150
198/198 [==============================] - 0s 1ms/step - loss: 1.1678e-06
Epoch 109/150
198/198 [==============================] - 0s 994us/step - loss: 1.0710e-06
Epoch 110/1

198/198 [==============================] - 0s 1ms/step - loss: 1.0372e-05
Epoch 33/150
198/198 [==============================] - 0s 1ms/step - loss: 1.1241e-05
Epoch 34/150
198/198 [==============================] - 0s 1ms/step - loss: 1.0042e-05
Epoch 35/150
198/198 [==============================] - 0s 1ms/step - loss: 1.0461e-05
Epoch 36/150
198/198 [==============================] - 0s 1ms/step - loss: 9.2952e-06
Epoch 37/150
198/198 [==============================] - 0s 1ms/step - loss: 1.0505e-05
Epoch 38/150
198/198 [==============================] - 0s 1ms/step - loss: 1.0914e-05
Epoch 39/150
198/198 [==============================] - 0s 1ms/step - loss: 9.5966e-06
Epoch 40/150
198/198 [==============================] - 0s 1ms/step - loss: 1.0135e-05
Epoch 41/150
198/198 [==============================] - 0s 1ms/step - loss: 9.8144e-06
Epoch 42/150
198/198 [==============================] - 0s 1ms/step - loss: 1.2151e-05
Epoch 43/150
198/198 [==============================] - 

Epoch 125/150
198/198 [==============================] - 0s 1ms/step - loss: 7.4241e-06
Epoch 126/150
198/198 [==============================] - 0s 1ms/step - loss: 7.4726e-06
Epoch 127/150
198/198 [==============================] - 0s 1ms/step - loss: 7.3150e-06
Epoch 128/150
198/198 [==============================] - 0s 1ms/step - loss: 7.1713e-06
Epoch 129/150
198/198 [==============================] - 0s 1ms/step - loss: 7.1214e-06
Epoch 130/150
198/198 [==============================] - 0s 1ms/step - loss: 6.8904e-06
Epoch 131/150
198/198 [==============================] - 0s 1ms/step - loss: 9.8484e-06
Epoch 132/150
198/198 [==============================] - 0s 1ms/step - loss: 7.3206e-06
Epoch 133/150
198/198 [==============================] - 0s 1ms/step - loss: 7.2616e-06
Epoch 134/150
198/198 [==============================] - 0s 1ms/step - loss: 7.4900e-06
Epoch 135/150
198/198 [==============================] - 0s 1ms/step - loss: 7.3276e-06
Epoch 136/150
198/198 [=========

198/198 [==============================] - 0s 947us/step - loss: 1.1533e-04
Epoch 60/150
198/198 [==============================] - 0s 989us/step - loss: 1.1523e-04
Epoch 61/150
198/198 [==============================] - 0s 980us/step - loss: 1.1545e-04
Epoch 62/150
198/198 [==============================] - 0s 978us/step - loss: 1.1491e-04
Epoch 63/150
198/198 [==============================] - 0s 970us/step - loss: 1.1557e-04
Epoch 64/150
198/198 [==============================] - 0s 974us/step - loss: 1.1498e-04
Epoch 65/150
198/198 [==============================] - 0s 966us/step - loss: 1.1511e-04
Epoch 66/150
198/198 [==============================] - 0s 979us/step - loss: 1.1584e-04
Epoch 67/150
198/198 [==============================] - 0s 966us/step - loss: 1.1529e-04
Epoch 68/150
198/198 [==============================] - 0s 978us/step - loss: 1.1547e-04
Epoch 69/150
198/198 [==============================] - 0s 965us/step - loss: 1.1579e-04
Epoch 70/150
198/198 [============

Arquivo com resultados gerados
